<a href="https://colab.research.google.com/github/shivansht5963/Python/blob/main/RNN_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
class DataReader:
    def __init__(self, file_path, seq_length):
        with open(file_path, 'r', encoding='utf-8') as f:
            self.data = f.read()
        self.chars = sorted(list(set(self.data)))
        self.vocab_size = len(self.chars)
        self.char_to_ix = {ch: i for i, ch in enumerate(self.chars)}
        self.ix_to_char = {i: ch for i, ch in enumerate(self.chars)}
        self.seq_length = seq_length
        self.pointer = 0

    def next_batch(self):
        if self.pointer + self.seq_length + 1 >= len(self.data):
            self.pointer = 0
        inputs = [self.char_to_ix[ch] for ch in self.data[self.pointer:self.pointer + self.seq_length]]
        targets = [self.char_to_ix[ch] for ch in self.data[self.pointer + 1:self.pointer + self.seq_length + 1]]
        self.pointer += self.seq_length
        return inputs, targets


In [ ]:
import numpy as np

class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        self.init_weights()

    def init_weights(self):
        self.Wxh = np.random.randn(self.hidden_size, self.vocab_size) * 0.01
        self.Whh = np.random.randn(self.hidden_size, self.hidden_size) * 0.01
        self.Why = np.random.randn(self.vocab_size, self.hidden_size) * 0.01
        self.bh = np.zeros((self.hidden_size, 1))
        self.by = np.zeros((self.vocab_size, 1))

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / np.sum(e_x)

    def loss_fun(self, inputs, targets, hprev):
        xs, hs, ys, ps = {}, {}, {}, {}
        hs[-1] = np.copy(hprev)
        loss = 0

        # Forward pass
        for t in range(len(inputs)):
            xs[t] = np.zeros((self.vocab_size, 1))
            xs[t][inputs[t]] = 1
            hs[t] = np.tanh(np.dot(self.Wxh, xs[t]) + np.dot(self.Whh, hs[t - 1]) + self.bh)
            ys[t] = np.dot(self.Why, hs[t]) + self.by
            ps[t] = self.softmax(ys[t])
            loss += -np.log(ps[t][targets[t], 0])

        # Backward pass
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        dhnext = np.zeros_like(hs[0])
        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1
            dWhy += np.dot(dy, hs[t].T)
            dby += dy
            dh = np.dot(self.Why.T, dy) + dhnext
            dhraw = (1 - hs[t] * hs[t]) * dh
            dbh += dhraw
            dWxh += np.dot(dhraw, xs[t].T)
            dWhh += np.dot(dhraw, hs[t - 1].T)
            dhnext = np.dot(self.Whh.T, dhraw)

        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)

        return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

    def sample(self, h, seed_ix, n):
        x = np.zeros((self.vocab_size, 1))
        x[seed_ix] = 1
        ixes = []
        for t in range(n):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            y = np.dot(self.Why, h) + self.by
            p = self.softmax(y)
            ix = np.random.choice(range(self.vocab_size), p=p.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[ix] = 1
            ixes.append(ix)
        return ixes
    def train(self, data_reader, iterations=10000):
        hprev = np.zeros((self.hidden_size, 1))
        for i in range(iterations):
            inputs, targets = data_reader.next_batch()
            loss, dWxh, dWhh, dWhy, dbh, dby, hprev = self.loss_fun(inputs, targets, hprev)

            # Update parameters
            for param, dparam in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by],
                                     [dWxh, dWhh, dWhy, dbh, dby]):
                param -= self.learning_rate * dparam

            if i % 250 == 0:
                sample_ix = self.sample(hprev, inputs[0], 100)
                generated = ''.join(data_reader.ix_to_char[ix] for ix in sample_ix)
                input_text = ''.join(data_reader.ix_to_char[ix] for ix in inputs)

                print(f"\nIteration: {i}, Loss: {loss:.4f}")
                print(f"Input: {input_text}")
                print(f"Generated Text: {generated}")


In [ ]:
seq_length = 25
data_reader = DataReader("interstellar_summary_condensed.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size, seq_length=seq_length, learning_rate=1e-3)
rnn.train(data_reader)



Iteration: 0, Loss: 95.7139
Input: Interstellar is a 2014 sc
Generated Text: liiqjan.B4HIqecSNk2 qviJijMdhovhJ2m,BceoMbtgSj0ToM2AigI,heTqkSp0IqsA4jnIwArqEgqkfHgh4uBtA2eJbf1mM,ks

Iteration: 250, Loss: 90.5309
Input: n future, Earth faces bli
Generated Text: V gdDgibjahJVBgjkCboBTdid urb.id 4E tiuotlpMtqdnM1no NhucHi.cM1jelbgg.mxSjjMhTSMdE bnm,l4hTmBDwfraxn

Iteration: 500, Loss: 84.3825
Input: ssion with scientists Bra
Generated Text:  y u ct. gqniJIxvdoonesg o,xpa D IIb 0INN eBjBe0uTdJirvoSnrortaHN l2y2ryysC0Hef4JAwtHRhrloCC .rIgiBR

Iteration: 750, Loss: 71.8979
Input: on across time with his d
Generated Text: 1hdcAft leosmrirRaesRe iiSoramt stfnBeia uuisV naso,iieRTawt,rrs srMtt esiHus r0ta, epe nnhAenwsaapa

Iteration: 1000, Loss: 72.0691
Input: ience fiction drama direc
Generated Text:  t ocie tt n ufideeowithtaeray0ibe pDi nthfo  eviahv  a Csstit slra Intoesats giips a ctfeEHgautiont

Iteration: 1250, Loss: 77.9675
Input: ght and famine. A group o
Generated Text: yhcds.maioeace.